Πρόσφατες επιθέσεις, όπως εκείνη στον πάροχο DNS Dyn (<a href="https://en.wikipedia.org/wiki/DDoS_attack_on_Dyn">wiki</a>) αποδεικνύουν ότι ένα από τα σημαντικότερα προβλήματα που αντιμετωπίζει το σύγχρονο Διαδίκτυο είναι εκείνο των botnets. Σε αυτά, ένας επιτιθέμενος συγκεντρώνει την υπολογιστική ισχύ που του είναι απαραίτητη για την εκδήλωση επιθέσεων DDoS ή/και άλλων κακόβουλων δραστηριοτήτων εγκαθιστώντας λογισμικό σε μεγάλο πλήθος από υπολογιστές (bots) που έχουν κενά ασφαλείας (π.χ. συσκευές Internet of Things - IoT).

Οι μολυσμένοι υπολογιστές (bots) διατηρούν διαύλους επικοινωνίας με το διαχειριστή του botnet (Command & Control Server) με σκοπό να λαμβάνουν εντολές και να αποστέλλουν πληροφορίες. Για το σκοπό αυτό εκμεταλλεύονται καθιερωμένα πρωτόκολλα, όπως το DNS με την παραγωγή μεγάλου πλήθους από domain names μέσω Domain Generation Algorithms (DGA's) που αλλάζουν διαρκώς για την επικοινωνία του bot με το διαχειριστή του, ώστε να αποφεύγεται ο εντοπισμός του Command & Control Server.

Τα ονόματα DNS που χρησιμοποιούνται από αλγορίθμους DGA μπορεί να είναι είτε τυχαία αλφαριθμητικά (π.χ. asdfasjkdfh8oawher8has.com) ή συνδυασμοί τυχαίων λέξεων που έχουν ληφθεί από κάποιο λεξικό (π.χ. school-doctor.com). Χρησιμοποιείται ένας μεγάλος αριθμός από τέτοια ονόματα, η πλειοψηφία των οποίων δεν έχουν κάποια αντιστοίχιση σε διεύθυνση IP και στοχεύουν στην απόκρυψη του Command & Control Server, επειδή οι αμυνόμενοι καλούνται να ελέγξουν κάθε ένα από τα ονόματα που παρατηρούν στο δίκτυό τους, σπαταλώντας χρόνο και πόρους. Επιπρόσθετα, η διεύθυνση IP του Command & Control Server αλλάζει πολύ συχνά (πολλές φορές σε μία μέρα), ώστε να αποφεύγεται ο εντοπισμός του ακόμα και όταν εντοπίζονται τα ονόματα DGA που οδήγησαν σε αυτόν.

<img src="https://raw.githubusercontent.com/nkostopoulos/StochasticsLabPublic/master/lab12/dga.png"></img>

<img src=""></img>

In [ ]:
import numpy
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt
import random

def add_word_distinct_chars(string, distinct_chars):
    for char in string:
        distinct_chars.add(char)
    return distinct_chars

def find_max_len(string, max_len):
    string_length = len(string)
    if string_length > max_len:
        max_len = string_length
    return max_len

def load_data(filename):
    dataset = []
    distinct_chars = set()
    max_len = 0
    currentIndex = 0
    diverse_labels = dict()
    with open(filename, "r") as fdr:
        for line in fdr:
            line = line.strip()
            general, label, name = line.split(",")
            name = name.split(".")[0]
            if label not in diverse_labels.keys():
                diverse_labels[label] = currentIndex
                currentIndex += 1
            distinct_chars = add_word_distinct_chars(name, distinct_chars)
            max_len = find_max_len(name, max_len)
            temp_list = []
            temp_list.append(name)
            temp_list.append(label)
            dataset.append(temp_list)
    random.shuffle(dataset)
    return dataset, distinct_chars, max_len, diverse_labels

def assign_index(chars):
    features = {}
    for index, char in enumerate(chars):
        features[char] = index
    return features

def convert_dataset_and_tokenize(dataset, features, max_len):
    for item_no, example in enumerate(dataset):
        name = example[0]
        label = example[1]
        tokenized = []
        padding_needed = max_len - len(name)
        for index in range(padding_needed):
            tokenized.append(0)
        for char in name:
            token = features[char]
            tokenized.append(token)
        example[0] = tokenized
        dataset[item_no] = example
    return dataset

def split_examples_labels(dataset):
    examples = [entry[0] for entry in dataset]
    labels = [entry[1] for entry in dataset]
    return examples, labels

def convert_labels_to_int(labels, diverse_labels):
    for index, label in enumerate(labels):
        labels[index] = diverse_labels[label]
    return labels

def build_model(max_features, max_len):
    model = Sequential()
    model.add(Embedding(max_features, 128, input_length = max_len))
    model.add(LSTM(128))
    model.add(Dropout(0.5))
    model.add(Dense(26, activation = 'softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    return model

def return_fold(examples, labels_int, num_folds, current_fold):
    interval = len(examples) // num_folds
    current_start = (current_fold - 1) * interval
    current_end = current_fold * interval
    X_test = examples[current_start:current_end]
    y_test = labels_int[current_start:current_end]
    X_train = examples[:current_start]
    y_train = labels_int[:current_start]
    X_train.extend(examples[current_end:])
    y_train.extend(labels_int[current_end:])
    return X_train, y_train, X_test, y_test

def make_predictions(model, X_test):
    y_pred = model.predict(X_test)
    y_pred = numpy.argmax(y_pred, axis = 1)
    return y_pred

def return_confusion_matrix(y_test, y_pred):
    return confusion_matrix(y_test, y_pred)

def return_classification_report(y_test, y_pred, diverse_names):
    target_names = []
    for item in diverse_names.keys():
        target_names.append(item)
    print(classification_report(y_test, y_pred, target_names = target_names))
    return None
    
if __name__ == "__main__":
    dataset, distinct_chars, max_len, diverse_labels = load_data("dga_domains_full.csv")
    dataset = dataset[0:100000]
    max_features = len(distinct_chars) + 1
    features = assign_index(distinct_chars)
    dataset = convert_dataset_and_tokenize(dataset, features, max_len)
    examples, labels = split_examples_labels(dataset)
    labels_int = convert_labels_to_int(labels, diverse_labels)

    acc_per_fold = []
    loss_per_fold = []
    number_of_folds = 5
    for fold in range(1, number_of_folds + 1):
        X_train, y_train, X_test, y_test = return_fold(examples, labels_int, number_of_folds, fold)
        model = build_model(max_features, max_len)
        print("Training for fold: ", fold)
        history = model.fit(X_train, y_train, batch_size = 128, epochs = 5, verbose = 1)
        scores = model.evaluate(X_test, y_test, verbose = 1)
        print(f'Score for fold {fold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
        acc_per_fold.append(scores[1] * 100)
        loss_per_fold.append(scores[0])
        y_pred = make_predictions(model, X_test)
        print(return_confusion_matrix(y_test, y_pred))
        return_classification_report(y_test, y_pred, diverse_labels)

    # == Provide average scores ==
    print('------------------------------------------------------------------------')
    print('Score per fold')
    for i in range(0, len(acc_per_fold)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
        print('------------------------------------------------------------------------')
        print('Average scores for all folds:')
        print(f'> Accuracy: {numpy.mean(acc_per_fold)} (+- {numpy.std(acc_per_fold)})')
        print(f'> Loss: {numpy.mean(loss_per_fold)}')
        print('------------------------------------------------------------------------')


Training for fold:  1
Epoch 1/5
625/625 [==============================] - 70s 104ms/step - loss: 1.4877 - accuracy: 0.5992
Epoch 2/5
625/625 [==============================] - 65s 104ms/step - loss: 0.9667 - accuracy: 0.7069
Epoch 3/5
625/625 [==============================] - 66s 105ms/step - loss: 0.8159 - accuracy: 0.7468
Epoch 4/5
625/625 [==============================] - 65s 103ms/step - loss: 0.7241 - accuracy: 0.7722
Epoch 5/5
625/625 [==============================] - 14s 22ms/step - loss: 0.5905 - accuracy: 0.8073
Score for fold 1: loss of 0.5904750823974609; accuracy of 80.73499798774719%
[[ 223    0  171    0    1    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    2    0]
 [   0  401    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    1    0    0    0    0    0    8    0    1]
 [  36    6 9752    1    8    5    3   38   17    9   24    0    6    0
    64    4    3    0    6    0   73   42    1    

Ο παραπάνω κώδικας επιλύει το πρόβλημα της ταξινόμησης ονομάτων DNS ως προς το αν αυτά τα ονόματα είναι καλόβουλα ή έχουν παραχθεί από Domain Generation Algorithms (DGA's). Συγκεκριμένα, ο κώδικας επιλύει ένα πρόβλημα multi-class classification, όπου τα ονόματα αντιστοιχίζονται είτε στην κατηγορία alexa, εάν είναι καλόβουλα ή στην αντίστοιχα κατηγορία malware στην οποία ανήκουν. Το μοντέλο που έχει χρησιμοποιηθεί για την ταξινόμηση των ονομάτων είναι το LSTM, ενώ για το κατάλληλο validation του μοντέλου έχει χρησιμοποιηθεί η μέθοδος K-fold cross validation με Κ = 5. Τα δεδομένα εκπαίδευσης και αξιολόγησης έχουν ληφθεί από <a href="https://github.com/chrmor/DGA_domains_dataset">εδώ</a> και χρησιμοποιούνται τα πρώτα 100000 ονόματα του αρχείου "dga_domains_full.csv". 

Μελετώντας τον κώδικα και εκτελώντας τον, να απαντήσετε στις ακόλουθες ερωτήσεις:
<ul>
<li>Γιατί το LSTM είναι κατάλληλο μοντέλο για την επίλυση του συγκεκριμένου προβλήματος;</li>
<li>Τι είναι η λίστα ονομάτων Alexa (<a href="https://en.wikipedia.org/wiki/Alexa_Internet">info</a>), ονόματα της οποίας έχουν χρησιμοποιηθεί για την κατηγορία των καλόβουλων ονομάτων;</li>
<li>Να διαλέξετε δύο οικογένειες malware από την κατηγορία dga, δηλαδή την κατηγορία με τα κακόβουλα ονόματα που αντιστοιχούν σε malware. Να αναζητήσετε τι προβλήματα δημιουργούν τα malware αυτά, π.χ. υποκλοπή τραπεζικών κωδικών, έναρξη επιθέσεων DDoS, κλπ.</li>
<li>Να περιγράψετε σύντομα τα βήματα που ακολουθεί το παραπάνω πρόγραμμα για την επίλυση του προβλήματος.</li>
<li>Ποιος είναι ο ρόλος του embedding layer και τι παραμέτρους δέχεται;</li>
<li>Να αναφέρετε λόγους για τους οποίους χρησιμοποιείται η μέθοδος K-fold cross validation.</li>
<li>Κατά τη χρήση της μεθόδου K-fold cross validation, ανά fold, να αναφέρετε πόσα examples χρησιμοποιούνται για την εκπαίδευση του μοντέλου και πόσα για το validation/testing.
<li>Δείτε <a href="https://en.wikipedia.org/wiki/Cross-validation_(statistics)">εδώ</a> μεθόδους που μπορεί να χρησιμοποιούν έναντι της μεθόδου K-fold cross validation. Να αναφέρετε μερικά πλεονεκτήματα και μειονεκτήματά τους.</li>
<li>Να αναλύσετε τα κριτήρια precision, recall και F1-score που εμφανίζονται στο classification report.</li>
<li>Σε ποιες κατηγορίες πετυχαίνουμε καλύτερα αποτελέσματα και σε ποιες χειρότερα; Γιατί πιστεύετε ότι παίρνουμε αυτά τα αποτελέσματα;</li>
</ul>